In [5]:

import sqlite3, json, os
import pandas as pd






## Loading the Static Data into a Database

This process is run once to initialize the database on local drive.

The database file can be accessed directly afterward without reloading the JSON files.

Existing tables are recreated (overwritten) during the initial load, but since the static data does not change, this has no effect on later usage.


In [6]:



db_path = "/home/fillies/Documents/moon/kilter/data/static.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()



In [7]:
cur.executescript("""
DROP TABLE IF EXISTS id_to_token;
DROP TABLE IF EXISTS token_to_id;
DROP TABLE IF EXISTS placements;
DROP TABLE IF EXISTS roles;
DROP TABLE IF EXISTS leds;
DROP TABLE IF EXISTS layouts;

CREATE TABLE id_to_token (id INTEGER PRIMARY KEY, token TEXT);
CREATE TABLE token_to_id (token TEXT PRIMARY KEY, id INTEGER);
CREATE TABLE placements (token_id INTEGER PRIMARY KEY, set_id INTEGER, x INTEGER, y INTEGER);
CREATE TABLE roles (role_id INTEGER PRIMARY KEY, name TEXT, color TEXT);
CREATE TABLE leds (led_id INTEGER, product_size_id INTEGER, position INTEGER,
                   PRIMARY KEY (led_id, product_size_id, position));
CREATE TABLE layouts (
  layout_name TEXT PRIMARY KEY,
  width INTEGER, height INTEGER,
  edge_left INTEGER, edge_right INTEGER,
  edge_bottom INTEGER, edge_top INTEGER
);
""")

In [8]:
def load_json(path):
    with open(path) as f:
        return json.load(f)
static_base = "/home/fillies/Documents/moon/kilter/static"

# id_to_token.json
data = load_json(os.path.join(static_base, "id_to_token.json"))
cur.executemany("INSERT INTO id_to_token VALUES (?, ?)", [(int(k), v) for k,v in data.items()])

# token_to_id.json
data = load_json(os.path.join(static_base, "token_to_id.json"))
cur.executemany("INSERT INTO token_to_id VALUES (?, ?)", [(k, int(v)) for k,v in data.items()])

# placements.json
data = load_json(os.path.join(static_base, "placements.json"))
cur.executemany("INSERT INTO placements VALUES (?, ?, ?, ?)", [(int(k), v["set"], v["x"], v["y"]) for k,v in data.items()])

# roles.json
data = load_json(os.path.join(static_base, "roles.json"))
cur.executemany("INSERT INTO roles VALUES (?, ?, ?)", [(int(k), v["name"], v["color"]) for k,v in data.items()])

# leds.json (nested arrays)
data = load_json(os.path.join(static_base, "leds.json"))
rows = []
for led_id, entries in data.items():
    for entry in entries:
        rows.append((int(led_id), entry["product_size_id"], entry["position"]))
cur.executemany("INSERT INTO leds VALUES (?, ?, ?)", rows)

# layoutInfo.json
data = load_json(os.path.join(static_base, "layoutInfo.json"))
cur.executemany("INSERT INTO layouts VALUES (?, ?, ?, ?, ?, ?, ?)",
                [(v["name"], v["width"], v["height"], v["edge_left"], v["edge_right"], v["edge_bottom"], v["edge_top"])
                 for v in data.values()])

conn.commit()
print("✅ All JSON files loaded into SQLite database!")

✅ All JSON files loaded into SQLite database!


In [10]:
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)
tables



,name
0,id_to_token
1,token_to_id
2,placements
3,roles
4,leds
5,layouts


In [11]:
for table in tables['name']:
    print(f"\n🔹 Table: {table}")
    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5;", conn)
    display(df)


🔹 Table: id_to_token


,id,token
0,0,<SOS>
1,1,<EOS>
2,2,<PAD>
3,3,p1447
4,4,p1073



🔹 Table: token_to_id


,token,id
0,<SOS>,0
1,<EOS>,1
2,<PAD>,2
3,p1447,3
4,p1073,4



🔹 Table: placements


...

,role_id,name,color
0,12,start,00FF00
1,13,middle,00FFFF
2,14,finish,FF00FF
3,15,foot,FFA500
4,20,start,00FF00



🔹 Table: leds


,led_id,product_size_id,position
0,1133,7,0
1,1133,10,0
2,1133,28,6
3,1134,7,1
4,1134,10,1



🔹 Table: layouts


,layout_name,width,height,edge_left,edge_right,edge_bottom,edge_top
0,8x12Home,1080,1755,24,120,0,156
1,7x10Small,1080,1473,28,116,36,156
2,7x10FullRideLEDKit,1080,1473,-44,44,24,144
3,7x10MainlineLEDKit,1080,1473,-44,44,24,144
4,7x10AuxiliaryLEDKit,1080,1473,-44,44,24,144


In [12]:
conn.close()